In [2]:
#To create dump of 2 chapter of the Translated version of ponniyinselvan. Thanks to Indira Neelamegam
import requests
from bs4 import BeautifulSoup
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
    'content-type': 'text/html; charset=UTF-8',
    'accept': 'text/html,*/*',
    'cookie': ""
}
urls=["https://tamilnation.org/literature/kalki/unicode/mp278a.htm","https://tamilnation.org/literature/kalki/unicode/mp278b.htm","https://tamilnation.org/literature/kalki/unicode/mp278c.htm","https://tamilnation.org/literature/kalki/unicode/mp278d.htm"]
words=""
for url in set(urls):
  xpage= requests.get(url, headers=headers)  
  prose=xpage.text   
  xsoup = BeautifulSoup( xpage.text)#html5lib
  body =xsoup.find_all("p")  
  for j in range(0,6):
    body[j].decompose()
  for p in body:
    words=words+" "+p.text

file = open("/content/corpus.txt", 'a') # write
file.write(words)
file.close()    

In [3]:
#configuring the OpenAI
!pip install openai
!pip install gpt_index==0.4.24
import openai
OPENAI_API_KEY="XXXXXXXXXXXXXXX"# get the API key from open AI
Org="XXXXXXXX"
openai.organization = Org
openai.api_key = OPENAI_API_KEY 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.5/590.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.4 MB/s eta 0:00:00


In [4]:
#Training the custom data with existing GPT Model
from gpt_index import SimpleDirectoryReader, GPTListIndex, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain.chat_models import ChatOpenAI

import sys
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def construct_index(directory_path):
    max_input_size = 4096
    num_outputs = 512
    max_chunk_overlap = 20
    chunk_size_limit = 600

    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index.save_to_disk('index.json')

    return index

def chatbot(input_text):
    index = GPTSimpleVectorIndex.load_from_disk('/content/index.json')
    response = index.query(input_text, response_mode="compact")
    return response.response



index = construct_index("/content")


In [5]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [6]:
print(chatbot("who is Vallavarayan Vandiya Devan?"))


Vallavarayan Vandiya Devan is a soldier who is travelling with the intention of reaching Kadamboor Fort before sunset. He is following the orders of his Prince to not enter into any skirmish until the assigned task is completed. He is on his way to meet Emperor Paranthaka Sundara Chozla and his beloved daughter, Pirati Kundavai.


In [7]:
print(chatbot("describe Kadamboor fort for 6 year old child"))


Kadamboor fort is a big and exciting place! It has a tall tower with a balcony at the top, where the lord of the fort stands and looks down at the people below. There are lots of guards who protect the fort and make sure no one can get in without permission. Inside the fort, there is a big palace with lots of rooms and courtyards. There are also lots of horses and soldiers who live and work there. It's a very busy place!


In [8]:
print(chatbot("what happen to Poonkuzlali in climax?"))


In the climax, Poonkuzlali agrees to go with the Prince to the shipwreck, where they collect grain and cook a meal. After eating, Poonkuzlali plans to sleep on the deck while the Prince waits on the bank with the elephant to guard her. In the morning, she will find her boat and leave.


In [ ]:
print(chatbot("Ponniyin Selvan married whom?"))


Ponniyin Selvan did not marry anyone in the context information provided.


In [ ]:
print(chatbot("Who is karikalan?"))


Karikalan is the Crown Prince of the Chozla Empire. He is married to a young girl in his old age and is the subject of a conspiracy. He is one of the powerful, distinguished and eminent personalities who attended the meeting mentioned in the context information.
